In [2]:
# load image from aws s3 bucket
import boto3
import tensorflow as tf
from PIL import Image
import numpy as np
s3 = boto3.resource('s3')
dataset = s3.Bucket('dalle2images')
image_paths = []
labels = []
bucket_name = 'dalle2images'
for obj in dataset.objects.all():
    if obj.key.endswith('.png') and obj.key.startswith('real'):
        image_paths.append(obj.key)
        labels.append(0)
    elif obj.key.endswith('.png') and obj.key.startswith('fake'):
        image_paths.append(obj.key)
        labels.append(1)

print("Number of images: %d" % len(image_paths))
print("Number of labels: %d" % len(labels))


ModuleNotFoundError: No module named 'tensorflow'

In [53]:
bucket_name = 'dalle2images'
images = []
for i in range(0, 100):
    # read image from s3 bucket
    img = s3.Object(bucket_name, image_paths[i])
    img = Image.open(img.get()['Body'])
    # convert image to numpy array
    img = np.array(img)
    # convert numpy array to tensor
    img = tf.convert_to_tensor(img, dtype=tf.float32)
    # add image to list
    images.append(img)

print("Number of images: %d" % len(images))

# convert list to tensor
images = tf.convert_to_tensor(images, dtype=tf.float32)


print("Tensor shape: %s" % images.shape)

# convert labels to tensor
labels = labels[:100]
labels = tf.convert_to_tensor(labels, dtype=tf.float32)
print("Tensor shape: %s" % labels.shape)

# create dataset
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
print(dataset)

Number of images: 100
Tensor shape: (100, 1024, 1024, 3)
Tensor shape: (100,)
<_TensorSliceDataset element_spec=(TensorSpec(shape=(1024, 1024, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>


In [55]:
# make the dataset ready for training
dataset = dataset.batch(32)
dataset = dataset.prefetch(1)
print(dataset)


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 1024, 1024, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


In [56]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1024, 1024, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Fit the model 
model.fit(dataset, epochs=3)

Epoch 1/3


2023-04-13 22:19:12.498901: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [100]
	 [[{{node Placeholder/_1}}]]
2023-04-13 22:19:13.913319: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.00GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-13 22:19:13.913373: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2023-04-13 22:19:23.120894: I tensorflow/compiler/xla/stream_executor/c

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_22/conv2d_25/Relu' defined at (most recent call last):
    File "/usr/local/python/3.10.4/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/local/python/3.10.4/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/codespace/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/codespace/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/codespace/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/codespace/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/local/python/3.10.4/lib/python3.

In [ ]:
# separate into train and test
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

# separate into train and validation
train_size = int(0.8 * len(train_dataset))
train_dataset = train_dataset.take(train_size)
validation_dataset = train_dataset.skip(train_size)

# print shapes
print("Train dataset shape: %s" % train_dataset.shape)
print("Validation dataset shape: %s" % validation_dataset.shape)

In [ ]:
# #resnet model

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers

# batchsize = 32
# epochs = 10

# data_augmentation = keras.Sequential(
#     [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
# )

# base_model = keras.applications.Xception(
#     weights="imagenet",  # Load weights pre-trained on ImageNet.
#     input_shape=(150, 150, 3),
#     include_top=False,
# )  # Do not include the ImageNet classifier at the top.

# # Freeze the base_model
# base_model.trainable = False

# # Create new model on top
# inputs = keras.Input(shape=(150, 150, 3))



In [ ]:
# import io
# import boto3
# import tensorflow as tf
# from PIL import Image
# import numpy as np


# def read_images_s3():
#     # Read images from S3
#     s3 = boto3.resource('s3')
#     bucket = s3.Bucket('dalle2images')
#     images = []
#     for obj in bucket.objects.all():
#         if obj.key.endswith('.png'):
#             image = Image.open(io.BytesIO(obj.get()['Body'].read()))
#             images.append(image)
#     print("Number of images: %d" % len(images))
#     return images

# def load_image_into_numpy_array(image):
#     # Convert image to numpy array
#     (im_width, im_height) = image.size
#     return np.array(image.getdata()).reshape(
#         (im_height, im_width, 3)).astype(np.uint8)

# def images():
#     # Read images from S3
#     images = read_images_s3()
#     # Convert images to numpy array
#     images_np = []
#     for image in images:
#         images_np.append(load_image_into_numpy_array(image))
#     # Convert numpy array to tensor
#     images_tensor = tf.convert_to_tensor(images_np, dtype=tf.float32)
#     print("Tensor shape: %s" % images_tensor.shape)
#     return images_tensor

